In [ ]:
pip install plotly

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
import numpy as np
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
import nltk

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
amRev = pd.read_csv("C:/Users/kahli/Desktop/TTS Files/datasets/Reviews.csv/Reviews.csv")
#Loading the data set

In [ ]:
amRev.head(5)
# Displaying the first 5 observations
#Text — This variable contains the complete product review information.
#Summary — This is a summary of the entire review.
#Score — The product rating provided by the customer.

In [ ]:
amRev.info()

In [ ]:
fig = px.histogram(amRev, x="Score")
fig.update_traces(marker_color="red",marker_line_color='rgb(8,48,107)',
                  marker_line_width=2)
fig.update_layout(title_text='Product Score')
fig.show()
# The counts of product score
# 1 being the worst 5 being the best
# As you can see based on the histogram most of the reviews are positive
# Score 1 - 52.268K
# Score 2 - 29.769K
# Score 3 - 42.64K
# Score 4 - 80.655K
# Score 5 - 363.122K

In [ ]:
newRev = amRev[amRev['Score'] != 3]
newRev.head()
# We remove 3 because it is a neutral score and will sway the results of a positive or negative score

In [ ]:
newRev['sentiment'] = newRev['Score'].apply(lambda rating : +1 if rating > 3 else 0)
newRev.head()
# For the sentiment column a 4 or 5 score is postive ( 1 )
# A 1 0r 2 score is negative ( 0 )

In [ ]:
newRev['sentimentt'] = newRev['sentiment'].replace({0 : 'negative'})
newRev['sentimentt'] = newRev['sentimentt'].replace({1 : 'positive'})
fig = px.histogram(newRev, x="sentimentt")
fig.update_traces(marker_color="cyan",marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5)
fig.update_layout(title_text='Product Sentiment')
fig.show()
# Split data set to graph
# Positive Sentiment = 443.777k
# Negative Sentiment = 82.037K

In [ ]:
def remove_punctuation(text):
    final = "".join(u for u in text if u not in ("?", ".", ";", ":",  "!",'"'))
    return final
newRev['Text'] = newRev['Text'].apply(remove_punctuation)
newRev = newRev.dropna(subset=['Summary'])
newRev['Summary'] = newRev['Summary'].apply(remove_punctuation)

newRev.head(5)
# Remove punctuation from the text and summary so our prediction can run smoothly without flagging certain characters as errors.



In [ ]:
ssRev = newRev[['Summary','sentiment']]
ssRev.head()
#Split the data tp get the summary and the sentiment


In [ ]:
index = newRev.index
newRev['random number'] = np.random.randn(len(index))

train = newRev[newRev['random number'] <= 0.8]
test = newRev[newRev['random number'] > 0.8]
# Splitting the data into an 80/20 split

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train['Summary'])
test_matrix = vectorizer.transform(test['Summary'])

# Takes the summary words and creates a data frame of just words that will be counted by number of occurance

In [ ]:
lr = LogisticRegression()
# Used for logistic regression

In [ ]:
X_train = train_matrix
X_test = test_matrix
y_train = train['sentiment']
y_test = test['sentiment']
# Used for independent variables

In [ ]:
lr.fit(X_train,y_train)
#Used to set the linear model

In [ ]:
predictions = lr.predict(X_test)
# Prediction model

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
new = np.asarray(y_test)
confusion_matrix(predictions,y_test)
#Test model accuracy and get confusion matrix

In [ ]:
print(classification_report(predictions,y_test))
#The Accuracy score measures how many labels the model got right out of the total number of predictions
#The Precision score is the number of correctly-identified members of a class divided by all the times the model predicted that class. 
#The Recall is the number of members of a class that the classifier identified correctly divided by the total number of members in that class.
#F1-Score or F-measure is an evaluation metric for a classification defined as the harmonic mean of precision and recall.
#It is a statistical measure of the accuracy of a test or model. 
#the value of F-measure(F1-score) reaches the best value at 1 and the worst value at 0. F1-score 1 represents the perfect accuracy and recall of the model.